# Импорт необходимых библиотек

In [6]:
!pip install --proxy 10.0.0.1:3128 gensim emoji pandas nltk pymorphy2

Using legacy 'setup.py install' for docopt, since package 'wheel' is not installed.
    Running setup.py install for docopt: started
    Running setup.py install for docopt: finished with status 'done'


You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec as wv
from gensim.models import KeyedVectors
import pymorphy2
from pymorphy2 import MorphAnalyzer
import emoji
import numpy as np

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [6]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

# Датасет ВК

## Загрузка датасета

In [27]:
colnames = ['comment',]

In [28]:
comments = pd.read_csv('datasets/vk.csv', names=colnames, header=0, sep='\t')

In [29]:
comments

,comment
0,['😭']
1,"['хранить', 'господь', 'неравнодушный', 'челов..."
2,"['евгений', 'еслиб', 'бог', 'такой', 'допускать']"
3,"['кирилл', 'полностью', 'согласный', '😢']"
4,"['кирилл', 'бог', 'путь', 'неисповедимый']"
...,...
50779,"['с', 'фамилия', '🙄']"
50780,"['наш', 'день', 'повезти', 'мужик', 'фамилия',..."
50781,"['чудо', 'чудной', 'диво', 'дивный']"
50782,"['пригреться', 'уснуть', 'свариться', 'лягушка..."


In [30]:
def str_to_list(i):
    new_list = eval(i)
    return new_list

In [31]:
comments['comment'] = comments['comment'].apply(str_to_list)

### Выделение эмодзи флагов

In [32]:
def split(s):
    return[char for char in s]

reg_ident = ['🇷', '🇺', '🇦', '🇸', '🇧', '🇨', '🇩', '🇪', '🇫', '🇬', '🇭', '🇮', '🇯', '🇰', '🇱', '🇲', '🇳', '🇴', '🇵', '🇶', '🇸', '🇹', 'v', '🇼', '🇽', '🇾', '🇿']

def get_flags(doc):
    for item in doc:
        new_item = []
        for c in split(item):
            if c in reg_ident:
                if len(new_item) < 2:
                    new_item.append(c)
        if len(new_item) == 2 and new_item[0] != new_item[1]:
            try:
                i = doc.index(item)
                doc[i] = new_item[0] + new_item[1]
            except ValueError:
                doc[i] = ''
    return doc

In [33]:
comments['comment'] = comments['comment'].apply(get_flags)

In [34]:
comments

,comment
0,[😭]
1,"[хранить, господь, неравнодушный, человек, 🙏, 🕊]"
2,"[евгений, еслиб, бог, такой, допускать]"
3,"[кирилл, полностью, согласный, 😢]"
4,"[кирилл, бог, путь, неисповедимый]"
...,...
50779,"[с, фамилия, 🙄]"
50780,"[наш, день, повезти, мужик, фамилия, но, суть,..."
50781,"[чудо, чудной, диво, дивный]"
50782,"[пригреться, уснуть, свариться, лягушка, так, ..."


### Добавление тегов из universal_tagset. Для этого нужно переименовать _taggers/universal_tagset/ru-rnс.map_ в _taggers/universal_tagset/ru-rnc-new.map_

In [36]:
def tagger(doc):
    tagged_tokens = []
    try:
        tagged_doc = nltk.pos_tag(doc, tagset='universal', lang = 'rus')
        for p in tagged_doc:
            if emoji.is_emoji(p[0]):
                token = p[0] + '_SYM'
            else:
                token = p[0] + '_' + p[1]
            tagged_tokens.append(token)
    except KeyError:
        print(doc)
        print('Тут ошибка формата!!')
    return(tagged_tokens)

In [37]:
comments_tagged = pd.DataFrame()

In [39]:
comments_tagged['tokens'] = comments['comment'].apply(tagger)

['ут', 'чк', 'м', 'ик', 'н', 'ки', 'тны', 'далее', 'м', 'нт', 'так', 'ы', 'ют', 'ук', 'ины', 'ближн', 'г', 'в', 'так', 'кит', 'так', 'л', 'ь', 'зный', 'далее', 'з', 'так', 'ль', 'тв', 'м', 'так', 'г', 'чт', 'к', 'мль', 'н', 'дит', 'п', 'далее', 'лп', 'м', 'п', 'нт', 'г', 'н', 'чт', 'ия', 'так', 'л', 'язвить', 'й', 'вн', 'шни', 'г', 'з', 'эт', 'зн', 'ч', 'так', 'чт', 'н', 'ци', 'н', 'льн', 'б', 'з', 'п', 'н', 'ть', 'ия', 'н', 'дит', 'п', 'далее', 'г', 'з', 'й', 'ия', 'ки', 'вл', 'ти', 'далее', 'лжный', 'п', 'инять', 'чны', 'м', 'ы', 'з', 'щит', 'так', 'ны']
Тут ошибка формата!!
['ут', 'чк', 'тны', 'далее', 'м', 'нт', 'ющи', 'ук', 'ины', 'ближн', 'г', 'в', 'так', 'кит', 'вид', 'так', 'ль', 'тв', 'так', 'так', 'м', 'чт', 'ия', 'н', 'дит', 'п', 'далее', 'п', 'так', 'льный', 'нт', 'л', 'м', 'м', 'ик', 'н', 'й', 'зв', 'дки', 'далее', 'м', 'нты', 'п', 'нт', 'г', 'н', 'г', 'ят', 'н', 'м', 'так', 'м', 'чт', 'широта', 'зн', 'ют', 'п', 'ия', 'ч', 'нь', 'мн', 'г', 'эт', 'явля', 'так', 'г', 'з', 'й

In [40]:
comments_tagged

,tokens
0,[😭_SYM]
1,"[хранить_VERB, господь_NOUN, неравнодушный_ADJ..."
2,"[евгений_ADJ, еслиб_NOUN, бог_NOUN, такой_PRON..."
3,"[кирилл_VERB, полностью_ADV, согласный_VERB, 😢..."
4,"[кирилл_VERB, бог_NOUN, путь_NOUN, неисповедим..."
...,...
50779,"[с_ADP, фамилия_NOUN, 🙄_SYM]"
50780,"[наш_PRON, день_NOUN, повезти_VERB, мужик_NOUN..."
50781,"[чудо_NOUN, чудной_ADJ, диво_ADV, дивный_ADJ]"
50782,"[пригреться_VERB, уснуть_VERB, свариться_VERB,..."


### Формирование списка списков

In [10]:
vk_data = comments['comment'].values.tolist()

In [11]:
vk_data[:5]

[['😭'],
 ['хранить', 'господь', 'неравнодушный', 'человек', '🙏', '🕊'],
 ['евгений', 'еслиб', 'бог', 'такой', 'допускать'],
 ['кирилл', 'полностью', 'согласный', '😢'],
 ['кирилл', 'бог', 'путь', 'неисповедимый']]

# Модель Word2Vec

## Инициализация готовой модели *()*

# Подготовка датасета для анализа тональности  *(дополнительно)* 

In [20]:
negative = ['😡', '👿', '😾', '😤', '😠', '🤬', '💀', '☠️']

In [21]:
sarcastic = ['💩', '🤡', '🙃', '🤢', '😈']  # Нужно дополнить

In [22]:
positive = ['😀', '😃', '😉']  # Нужно дополнить